In [ ]:
import re
import zlib
import base64
import datetime
import numpy as np
import pandas as pd
import json
import configparser
from utils import *
from sys import platform
from elasticsearch import Elasticsearch

In [ ]:
cf = configparser.ConfigParser()
cf.read('config/config.cfg')

env = 'DEVELOP'
if 'win' in platform:
    env = 'DEVELOP'
elif 'linux' in platform:
    env = 'PRODUCT'
    
class EsCtrl(object):
    def __init__(self):
        self.es_ctrl = Elasticsearch(cf['ENV_'+env]['ADDR'], ca_certs=cf['ELASTICSEARCH']['CA_CERTS'])

    def query_index_logs(self, index):
        # query = {
        #     "match": {
        #         "trace": "com_ericsson_trithread:INFO"
        #     }
        # }
        #data = self.es_ctrl.search(index=index, query=query, scroll='1s', size=10000)
        data = self.es_ctrl.search(index=index, scroll='1s', size=10000)
        sid = data['_scroll_id']
        scroll_size = len(data['hits']['hits'])
        res = []
        while scroll_size > 0:
            # Before scroll, process current batch of hits
            res.extend(data['hits']['hits'])
            data = self.es_ctrl.scroll(scroll_id=sid, scroll='1s')
            # Update the scroll ID
            sid = data['_scroll_id']
            # Get the number of results that returned in the last scroll
            scroll_size = len(data['hits']['hits'])
        return res

    def query_indices(self):
        res = []
        for key in self.es_ctrl.indices.get_alias().keys():
            if len(key) > 0:
                if '.analyzed_' in key:
                    res.append(key.replace('.analyzed_', ''))
        return res

    def is_exists(self, index):
        return self.es_ctrl.indices.exists(index=index)

    def count_index(self, index):
        return self.es_ctrl.count(index=index)['count']

    def store_index(self, index, data):
        data = deflate_and_base64_encode(json.dumps(data).encode('utf-8'))
        return self.es_ctrl.index(index=index, body={'content': data})

    def query_index(self, index):
        data = self.es_ctrl.search(index=index)
        data = json.loads(decode_base64_and_inflate(data['hits']['hits'][0]['_source']['content']))
        return data

In [ ]:
# 1.值范围 2.相似线段形态 3.highlight落在区间内 4.方差大

In [12]:
import json
import uuid
branch = ['A','B','C','D','E','F','G','H','I','J','K','L']
config = {'search':[], 'insight':[], 'chart':[], 'statistic':[]}
# for index in range(12):
#     item = {}
#     item['alias'] = 'search_branch' + str(branch[index])
#     item['desc'] = 'Branch ' + str(branch[index]) + ' common use KeyValue and Mark'
#     item['exp_search'] = '(txlProcBranch'+str(branch[index])+'|TxBranchCtrl'+str(branch[index])+').*(txAtt|linearization fault|external fault)'
#     item['exp_extract'] = [
#                         "{}[{timestamp:ti}] {}txAtt:{txAtt:d}, {}torTemperature:{torTemperature:d} {}avgIMpa0:{avgIMpa0:d} {}",
#                         "{}[{timestamp:ti}] {}"
#                         ]
#     item['exp_mark'] = [{"alias":"LF","exp":"linearization fault","color":"#f00000"}]
#     item['is_case_sensitive'] = True
#     item['forward_rows'] = 0
#     item['backward_rows'] = 0
#     config['search'].append(item)

for index in range(12):
    item = {}
    item['alias'] = 'insight_branch' + str(branch[index])
    item['desc'] = 'Branch Insight' + str(branch[index])
    item['exp_search'] = '(txlProcBranch'+str(branch[index])+'|TxBranchCtrl'+str(branch[index])+')'
    item['exp_extract'] = "{}[{timestamp:ti}]{}, msg = {msg}"
    item['exp_mark'] = {"alias":"LF","exp":"linearization fault","color":"#f00000"}
    item['is_case_sensitive'] = True
    item['forward_rows'] = 0
    item['backward_rows'] = 0
    config['insight'].append(item)

# for index in range(12):
#     item = {}
#     item['alias'] = 'statistic_branch' + str(branch[index])
#     item['desc'] = 'Branch ' + str(branch[index]) + ' statistic txAtt'
#     item['code'] = "self.result = self.text_analysis_model.alias_data['" + 'search_branch' + str(branch[index]) + "'].res_key_value['txAtt']['global_index']"
#     config['statistic'].append(item)
    
json_object = json.dumps(config)
with open("D:\\projects\\ericsson_flow\\new_files\\config2.txt", "w") as outfile:
    outfile.write(json_object)

In [ ]:
import re
from parse import parse

str1 = 'BXP_3: [2022-11-23 22:04:44.572174570] (+0.000333470) radio6626 com_ericsson_trithread:INFO: { cpu_id = 1 }, { process = "txlProcBranchB", fileAndLine = "dpdController.cc:1887", msg = "txAtt:289, txAttPeak:0, dpGainLoopEnable:true, dpGainCtrlType:DSA_AD_TXFE, torTemperature:495 (0.1C), torGainBackoff:0 (0.01dB), torGainLin:3.49543(0.01dB), torStepBit:6, cc0Ctrl1=0x00000118 , avgIMpa0:1980 [mAmp]" }'
str2 = 'BXP_3: [2022-12-10 15:55:26.739019220] (+0.000025340) radio6626 com_ericsson_trithread:INFO: { cpu_id = 1 }, { process = "TxBranchCtrlB", fileAndLine = "txChangeCycleHelper.cc:264", msg = "Txl branch J restart due to txL linearization fault!" }'
# str1 = "BXP_2: [221120 164014] 27: PA measured values for driver name: DpaVddSv:7; value: 26992; branch Id: 7"
# exp = "\[%{TIMESTAMP:time}\] \(%{STRING:cost}\) "
# exp = "\[%{TIMESTAMP:time}\] %{DROP:tmp}Pma:%{FLOAT:Pma0}\[%{DROP:tmp1}DpdPma:%{FLOAT:DpdPma0}\[%{DROP:tmp2}Pmb:%{FLOAT:Pmb}, TorPmb:%{FLOAT:TorPmb0}\[%{FLOAT:TorPmb1} %{FLOAT:TorPmb2}\] "
exp_search = '(txlProcBranchB|TxBranchCtrlB).*(txAtt|linearization fault)'
exp_extract = '{}[{timestamp:ti}] {}txAtt:{txAtt:d}, {}torTemperature:{torTemperature:d} {}avgIMpa0:{avgIMpa0:d} {}'
r_search = re.findall(exp_search, str1)
r_extract = parse(exp_extract, str1)
print(r_search)
print(r_extract.named)

In [ ]:
aaa = 111
class A():
    def __init__(self, data):
        self.a = 1
        self.b = 2
        self.__dict__.update(data)
        
    def test(self, num):
#         aaa = 222
        print(aaa)

class B(A):
    def __init__(self):
        super().__init__()
        self.c = 3
        self.d = {'ins': ''}
        
a = A({'a':3, 'b':4})
# b = B()
# data = {'a':a, 'b':b}

In [ ]:
class A:
    def __init__(self):
        self.a = 1

class B(A):
    def __init__(self):
        super().__init__()
        self.b = 2
        self.c = C()

        
    def set_attr(self):
        exec("self.a['a']['b']['c'] = 'aaa'")
        
class C():
    def __init__(self):
        self.c = 3

c = B()
c.a = {'a':{'b':{'c':'foo', 'e':12 , 'f':True}}}

b = B()
b.a = {'a':{'b':{'c':42, 'd':33, 'f':False}}}
b.b = 3

In [ ]:
from deepdiff import DeepDiff

item1 = {'a':{'b':{'c':'foo', 'e':12 , 'f':True}}}
item2 = {'a':{'b':{'c':42, 'd':33, 'f':False}}}
DeepDiff(item1, item2)

In [ ]:
DeepDiff(c.a, b.a)

In [9]:
str1 = '/a/b/c/d'
str1.split('/')

['', 'a', 'b', 'c', 'd']

In [ ]:
import re
import json
from text_analysis import TextAnalysisModel

text_analysis_model = await TextAnalysisModel('parallel', mode = 'test')

file_path = ['D:\\Projects\\ericsson_flow\\new_files\\E55G948878_LE_SARONGGE_BXP_6_telog.log']
config_path = 'D:\\Projects\\ericsson_flow\\new_files\\config2.txt'
config = [config_path, json.dumps(json.load(open(config_path)))]

await text_analysis_model.file_container_model.on_new_file('', file_path)
await text_analysis_model.file_container_model.on_load_config('', config)

for key in text_analysis_model.alias_data.keys():
    print(key, len(text_analysis_model.alias_data[key].outlier))

In [ ]:
for index, item in enumerate(text_analysis_model.alias_data['insight_branchJ'].outlier):
    if item['abnormal_type'] == 'AbnormalUpPulse':
        print(index, item['timestamp'], item['desc'], re.findall('msg = "(.*?)"', item['origin']))

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

import en_core_web_sm
nlp = en_core_web_sm.load()

In [ ]:
def self_clean_special_symbols(text, symbol):
    for ch in ['.', '_','-']:
        if ch in text:
            text = text.replace(ch,symbol)
    text = re.sub(r'\d+', '', text)
    return re.sub(symbol+"+", symbol, text)
        
def camel_case_split(s):
    idx = list(map(str.isupper, s))
    # mark change of case
    l = [0]
    for (i, (x, y)) in enumerate(zip(idx, idx[1:])):
        if x and not y:  # "Ul"
            l.append(i)
        elif not x and y:  # "lU"
            l.append(i+1)
    l.append(len(s))
    # for "lUl", index of "U" will pop twice, have to filter that
    return [s[x:y].lower() for x, y in zip(l, l[1:]) if x < y]

special_words = set(['timeout', 'fault', 'error', 'abn', 'shutdown', 'deactivate' , 'activate'])
for index, item in enumerate(text_analysis_model.alias_data['insight_branchI'].outlier):
    if item['abnormal_type'] == 'UniquePrint':
        t = self_clean_special_symbols(item['value'], ' ')
        tmpwords = t.split(' ')
        words = []
        [words.extend(camel_case_split(word)) for word in tmpwords]
        doc = nlp(' '.join(words))
        pos = [w.pos_ for w in doc]
        flag = True if len(set(pos).intersection(set(['VERB', 'AUX']))) > 0 else False
        flag = True if len(set(words).intersection(special_words)) > 0 else flag
        if flag == True:
            print(index, flag, item['value'], re.findall('msg = "(.*?)"', item['origin']), set(pos))
#         print(index, item['abnormal_type'], item['value'], re.findall('msg = "(.*?)"', item['origin']))
        
# camel_case_split('IPaddress')
# doc = nlp("77868817us")
# print([(w.text, w.pos_) for w in doc]) 39

In [ ]:
re.sub(r'\d+', '',"hello 42 I'm a 32 string 30")

In [ ]:
doc = nlp("Training Signal sequence successfully done Activated")
print([(w.text, w.pos_) for w in doc])

In [1]:
import re
import json
from text_analysis import TextAnalysisModel

text_analysis_model = await TextAnalysisModel('parallel', mode = 'test')

dir_path = 'D:\\projects\\ericsson_flow\\batch_test'
config = 'D:\\projects\\ericsson_flow\\new_files\\config2.txt'

await text_analysis_model.file_container_model.batch_insight_model.new(dir_path, config)

D:\projects\ericsson_toolsets\src\python\text_analysis.py:1352: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.result = self.result.append(sample, ignore_index=True)


Finish : E55G948878_LE_SARONGGE_BXP_6_telog.log
Finish : E55H060478_LE_JLADANSUKAHATI_CBX_PL_BXP_6_telog.log


D:\projects\ericsson_toolsets\src\python\text_analysis.py:1352: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.result = self.result.append(sample, ignore_index=True)


In [7]:
text_analysis_model.file_container_model.batch_insight_model.result.loc[1, 'resOutlier']

,name,type,abnormal_type,global_index,search_index,timestamp,value,desc,origin
0,txlSrv.cc,float,AbnormalUpPulse,15632,222,2022-12-10 18:45:25.591580,txlSrv.cc_AbnormalUpPulse,txlSrv.cc: 564.0 --> 626.0 --> 564.0,BXP_6: [2022-12-10 18:45:25.591580320] (+0.000...
1,10,float,AbnormalUpPulse,15600,204,2022-12-10 18:45:23.760677,10_AbnormalUpPulse,10: 1.0 --> 7.0 --> 1.0,BXP_6: [2022-12-10 18:45:23.760677490] (+0.010...
2,avgIMpa0,float,AbnormalUpPulse,15585,189,2022-12-10 18:45:23.746493,avgIMpa0_AbnormalUpPulse,avgIMpa0: 0.0 --> 490.0 --> 0.0,BXP_6: [2022-12-10 18:45:23.746493610] (+0.000...
3,txAtt,float,AbnormalUpPulse,15584,188,2022-12-10 18:45:23.746474,txAtt_AbnormalUpPulse,txAtt: 341.0 --> 2500.0 --> 341.0,BXP_6: [2022-12-10 18:45:23.746474840] (+0.001...
4,list,float,AbnormalUpPulse,12593,141,2022-12-10 18:45:17.598833,list_AbnormalUpPulse,list: 0.0 --> 9.0 --> 0.0,BXP_6: [2022-12-10 18:45:17.598833690] (+0.000...
5,fbsId,float,AbnormalUpPulse,10982,12,2022-12-10 18:45:17.086798,fbsId_AbnormalUpPulse,fbsId: 1.0 --> 2.0 --> 1.0,BXP_6: [2022-12-10 18:45:17.086798670] (+0.000...
6,carrierId,float,AbnormalUpPulse,10982,12,2022-12-10 18:45:17.086798,carrierId_AbnormalUpPulse,carrierId: 784.0 --> 796.0 --> 784.0,BXP_6: [2022-12-10 18:45:17.086798670] (+0.000...


In [4]:
text_analysis_model.file_container_model.batch_insight_model.samples

,txAtt_AbnormalUpPulse,avgIMpa0_AbnormalUpPulse,10_AbnormalUpPulse,fbsId_AbnormalUpPulse,carrierId_AbnormalUpPulse,list_AbnormalUpPulse,txlSrv.cc_AbnormalUpPulse,label
0,1,1,1,0.0,0.0,0.0,0.0,0
1,1,1,1,1.0,1.0,1.0,1.0,1


In [ ]:
tmp = text_analysis_model.file_container_model.batch_insight_model.result
res = tmp.loc[(tmp['fileName'] == 'E55H060478_LE_JLADANSUKAHATI_CBX_PL_BXP_6_telog.log'), :].reset_index(drop=True).loc[0, :]
dict(res)

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pandas as pd

X = pd.DataFrame([[0,22],[0,22],[0,22],[0,255],[0,255],[0,255]])
cost =[]
for i in range(1, 4):
    KM = KMeans(n_clusters = i, max_iter = 500)
    KM.fit(X)

    # calculates squared error
    # for the clustered points
    cost.append(KM.inertia_)
    
# plot the cost against K values
plt.plot(range(1, 4), cost, color ='g', linewidth ='3')
plt.xlabel("Value of K")
plt.ylabel("Squared Error (Cost)")
plt.show() # clear the plot

In [14]:
str1 = '/a/b/c/d'
'/'.join(str1.split('/')[0:-1])

'/a/b/c'

In [2]:
a = []
a.extend(range(1,5))

In [4]:
a = sorted(set([3,2,1,1,5]))
a

[1, 2, 3, 5]